In [1]:
import videoio
from PIL import Image
import numpy as np
from sklearn import datasets
import tensorflow as tf
import os
import uuid

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

## Create path for features

In [3]:
def create_path(path):
    if not os.path.exists(path):
        os.mkdir(path)

In [11]:
DATASET_PATH_TRAIN = '/home/dantas/kinetics/train'
DATASET_PATH_VALIDATION = '/home/dantas/kinetics/validation'
DATASET_PATH_TEST = '/home/dantas/kinetics/test'

In [12]:
FEATURES_PATH_TRAIN = '/media/DATA/features/train'
FEATURES_PATH_VALIDATION = '/media/DATA/features/validation'
FEATURES_PATH_TEST = '/media/DATA/features/test'

In [13]:
create_path('/media/DATA/features')
create_path(FEATURES_PATH_TRAIN)
create_path(FEATURES_PATH_VALIDATION)
create_path(FEATURES_PATH_TEST)

## Create model

In [7]:
inception_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')

## Extract features for `train` and `validation`

In [8]:
def generate_and_save_features(path, dataset, model):
    for target, item in zip(dataset.target, dataset.filenames):
        create_path(f'{path}/{dataset.target_names[target]}')
        
        frames = videoio.videoread(item, output_resolution=(299, 299))
        frames = np.array(frames)

        video_features = np.empty((0, 8, 8, 2048))
        for chunk in np.array_split(frames, 3):
            chunk_features = model(chunk)
            video_features = np.concatenate((video_features, chunk_features))
        
        name = uuid.uuid4()
        np.save(f'{path}/{dataset.target_names[target]}/{name}.npy', video_features)

### Create dataset

In [14]:
dataset_train = datasets.load_files(DATASET_PATH_TRAIN)
dataset_validation = datasets.load_files(DATASET_PATH_VALIDATION)

### Extract and save features

In [15]:
generate_and_save_features(FEATURES_PATH_TRAIN, dataset_train, inception_model)

In [16]:
generate_and_save_features(FEATURES_PATH_VALIDATION, dataset_validation, inception_model)

## Extract features for `test`

In [17]:
for item in os.listdir(DATASET_PATH_TEST):
    if item != 'tmp':
        frames = videoio.videoread(f'{DATASET_PATH_TEST}/{item}', output_resolution=(299, 299))
        frames = np.array(frames)

        video_features = np.empty((0, 8, 8, 2048))
        for chunk in np.array_split(frames, 3):
            chunk_features = inception_model(chunk)
            video_features = np.concatenate((video_features, chunk_features))

        name = uuid.uuid4()
        np.save(f'{FEATURES_PATH_TEST}/{name}.npy', video_features)